# Initialization

In [1]:
#Import all the UTILITY functions
from UTILITY_quickstart import *

#Look at the signature and documentation for initializeTao()
?initializeTao

Signature:
initializeTao(
    filePath=None,
    lastTrackedElement='end',
    csrTF=False,
    inputBeamFilePathSuffix=None,
    numMacroParticles=None,
    loadDefaultLatticeTF=True,
    defaultsFile=None,
    runImpactTF=False,
    scratchPath=None,
    randomizeFileNames=False,
    transverseWakes=False,
    **kwargs,
)
Docstring:
Initialize a tao object from PyTao

Specify the active file path

Specify the number of macroparticles

Either load a particular lattice or load the golden lattice

Have the option to randomize file paths to facilitate parallelization

Optionally enable transverse wakefields
File:      ~/Documents/SLAC/FACET2-S2E/UTILITY_quickstart.py
Type:      function

In [2]:
#Load a default configuration
importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline.yml")

#Initialize tao object
tao = initializeTao(
    inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    numMacroParticles = 1e4,
    csrTF = False
)

#Set lattice to the loaded config
setLattice(tao, **importedDefaultSettings)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-S2E
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using /Users/nmajik/Documents/SLAC/FACET2-S2E/setLattice_configs/defaults.yml
Number of macro particles = 10000.0
Beam created, written to /Users/nmajik/Documents/SLAC/FACET2-S2E/beams/activeBeamFile.h5, and reinit to tao


# Track beam

In [5]:
#Track the beam from start to end
trackBeam(tao, **importedDefaultSettings, verbose = True)

Loaded /Users/nmajik/Documents/SLAC/FACET2-S2E/beams/activeBeamFile.h5
Set track_start = L0AFEND, track_end = BEGBC20
Set track_end = BEGBC14_1
Tracking!
Working on file: /Users/nmajik/Documents/SLAC/FACET2-S2E/beams/patchBeamFile.h5
['/', '/particles', '/particles/electron', '/particles/electron/momentum', '/particles/electron/momentum/x', '/particles/electron/momentum/y', '/particles/electron/momentum/z', '/particles/electron/particleStatus', '/particles/electron/position', '/particles/electron/position/x', '/particles/electron/position/y', '/particles/electron/position/z', '/particles/electron/time', '/particles/electron/weight']
[1.30573547e-06 1.30573366e-06 1.30573472e-06 ... 1.30573377e-06
 1.30573452e-06 1.30573397e-06]
Beam centered at BEGBC14 written to /Users/nmajik/Documents/SLAC/FACET2-S2E/beams/patchBeamFile.h5
Loaded /Users/nmajik/Documents/SLAC/FACET2-S2E/beams/patchBeamFile.h5
Set track_start = BEGBC14_1, track_end = BEGBC20
Tracking!
trackBeam() exiting


In [ ]:
#Get the beam at element "DTOTR"
P =  getBeamAtElement(tao, "DTOTR")

#Plot the x-y projection of the beam at that location
display(plotMod(P, 'x', 'y',  bins=200));
plt.clf()

print(P)

# Change a quad setting and retrack

In [ ]:
#Set the strength of a final focus quad named Q0FF to zero kilogauss 
setQuadkG(tao, "Q0FF", 0)

#Track the beam through the now modified lattice
trackBeam(tao, **importedDefaultSettings)

In [ ]:
P =  getBeamAtElement(tao, "DTOTR")

display(plotMod(P, 'x', 'y',  bins=200));

In [ ]:
#Get the beam at the element MFFF, at the exit of BC20
P =  getBeamAtElement(tao,"MFFF")

#Plot the longitudinal phase space
display(plotMod(P, 'delta_t', 'pz',  bins=200));

# Change L2 phase and retrack

In [ ]:
#Change the phase of L2 to -10 degrees
setLinacPhase(tao, "L2", -10)
trackBeam(tao, **importedDefaultSettings)

In [ ]:
P =  getBeamAtElement(tao, "MFFF")


display(plotMod(P, 'delta_t', 'pz',  bins=200));
plt.clf();

# Example phase scan

In [ ]:
#There are more efficient ways to do this, but here's a toy example of doing a phase scan and looking at how it impacts the final beam energy

results = {"phase": [], "energy": []}
for L2PhaseSet in np.arange(-40, 10, 5):
    setLinacPhase(tao, "L2", L2PhaseSet)
    trackBeam(tao)
    P =  getBeamAtElement(tao, "MFFF")
    #results.append([L2PhaseSet, P["mean_energy"]])
    results["phase"].append(L2PhaseSet)
    results["energy"].append(P["mean_energy"])
    print([L2PhaseSet, P["mean_energy"]])

In [ ]:
plt.scatter(results["phase"], results["energy"])

# Adding title and labels
plt.title('Scatter Plot of Mean Energy vs L2 Phase Set')
plt.xlabel('L2 Phase Set')
plt.ylabel('Mean Energy')

# Display the plot
plt.show()